In [43]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [44]:
def buyandhold(data_frame, is_plot=False):
  """
  Buy and Hold strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """

  close = data_frame['Adj Close']
  status = 0
  signal = []
  for k in range(close.shape[0]):
    if k == 0:
      signal.append(1)
    else:
      signal.append(0)
  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)    
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","Buy", "Sell"])
    plt.title("Buy and Hold")
    plt.grid()
  return signal


In [45]:
def bollinger(data_frame, is_plot=False):
  """
  Bollinger Band strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  close = data_frame['Adj Close']
  win = 10 
  ma = close.rolling(win).mean()
  std = close.rolling(win).std()
  up = ma + std
  low = ma - std  
  status = 0
  signal = []
  for k in range(close.shape[0]):
    closek = close[k]
    if (closek >  up[k]) and (status == 0): # buy
      signal.append(1)
      status = 1
    elif (closek < low[k]) and (status == 1): # sell
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)
    plt.plot(up)
    plt.plot(low) 
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","High", "Low", "Buy", "Sell"])
    plt.title("Bollinger Bands")
    plt.grid()
  return signal


In [46]:
def rsi(data_frame, is_plot=False):
  """
  Relative Signal Strength:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  window_length= 14
  upv = 70
  lowv= 40
  close = data_frame['Adj Close']
  delta = close.diff(1) 
 
  up, down = delta.copy(), delta.copy()
  up[up < 0] = 0
  down[down > 0] = 0
  roll_up1 = up.ewm(span=window_length).mean()
  roll_down1 = down.abs().ewm(span=window_length).mean()
  # Calculate the RSI based on EWMA
  RS1 = roll_up1 / roll_down1
  RSI1 = 100.0 - (100.0 / (1.0 + RS1))
  status = 0
  signal = []
  for k in range(close.shape[0]):
    rsk = RSI1[k]
    if (rsk < lowv) and (status == 0): # buy
      signal.append(1)
      status = 1
    elif (rsk > upv) and (status == 1): # sell
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]

  if is_plot:  
    plt.figure(figsize=(15,10))
    plt.subplot(2,1,1)
    plt.plot(close)
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","Buy", "Sell"])
    plt.grid()
    plt.title("RSI")
    plt.subplot(2,1,2)
    plt.plot(RSI1)
    plt.plot(RSI1.index, upv*np.ones((close.shape[0],)),"b--")
    plt.plot(RSI1.index, lowv*np.ones((close.shape[0],)),"b--")
    plt.legend(['RSI', "SELL Band", "BUY Band"])
    plt.grid()
    plt.title("RSI Value")
  return signal


In [47]:
def movingavg(data_frame, is_plot=False):
  """
  Moving Average crossing strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  long_win = 25
  short_win = 10
  close = data_frame['Adj Close'] 
  mu_long = close.rolling(long_win).mean()
  mu_short = close.rolling(short_win).mean()
  signal = []
  status = 0
  for k in range(close.shape[0]):
    if ( mu_short[k] > mu_long[k]) and (status ==0):
      signal.append(1)
      status = 1
    elif (mu_short[k] < mu_long[k]) and (status ==1):
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)    
    plt.plot(mu_long)
    plt.plot(mu_short)
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close",f"MA({long_win})", f"MA({short_win})", "Buy", "Sell"])
    plt.title("Moving Average Crossing")
    plt.grid()
  return signal


In [48]:
def techicalTrading(stock_info, # dataframe of a stock
                    indicator, # trade indicator
                    initial_money=1000000 #Starting money
                    ):
  """
  Trader simulation:
  Inputs: 
        stock_info: stock price data frames
        indicator: time-series indicator: 1 for Buy, -1 for sell, 0 for nothing
        initinal_money: initial money in the port. default 1M 
  output:
        wealth: time-series money in the port
  """

  num_dates, _ = stock_info.shape
  status = "NONE" # Start with no poistion
  stock= 0
  money = initial_money
  wealth = []
  close = stock_info['Adj Close']
  for k in range(num_dates):
    indicatork = indicator[k]
    if not np.isnan(indicatork): # if it is not NA
      if (indicatork > 0) and ( status == "NONE"):
        stock = money/close.values[k]
        #print(f"Buy at {close.values[k]}")
        money = 0
        status = "HOLD"
      elif (indicatork < 0) and ( status == "HOLD"):
        money = stock * close.values[k]
        stock =  0
        #print(f"Sell at {close.values[k]}")
        status = "NONE"
    cur_wealth = money + stock * close.values[k]
    wealth.append(cur_wealth)
  wealth = pd.Series(data=wealth, index=stock_info.index)  
  return wealth 



In [49]:
set50_list = ["ADVANC","AOT","AWC","BBL","BDMS","BEM","BGRIM","BH","BJC","BPP","BTS","CBG","CPALL","CPF","CPN","CRC","DTAC","EA","EGCO","GLOBAL","GPSC","GULF","HMPRO","INTUCH","IRPC","IVL","KBANK","KTB","KTC","LH","MINT","MTC","OSP","PTT","PTTEP","PTTGC","RATCH","SAWAD","SCB","SCC","SCGP","TISCO","TMB","TOA","TOP","TRUE","TTW","TU","VGI","WHA"]
stocks = pd.Series(set50_list)+'.bk'
df = data.DataReader(stocks, data_source="yahoo", start='2020-01-01', end='2020-12-31')

Question 1: Use dataset of SET50 list in 2020 (https://www.set.or.th/th/market/files/constituents/SET50_100_H2_2020_revised.pdf) Compute the return in 2020 from 2020-01-01 to 2020-12-31. Find out

1. Average annual return 0.41% and annual STD 6.40 for buy and hold

2. Average annual return 0.00% and annual STD 24.05 for moving average crossing

3. Average annual return 0.00% and annual STD 30.15 for Bollinger Band

4. Average annual return 0.41% and annual STD 21.32 for RSI

In [50]:
BNH = buyandhold(df, is_plot=False)
avgBNH = np.mean(BNH)
stdBNH = np.std(BNH)

print(f"Average annual return is {avgBNH*100.0:2.2f}%")
print(f"Average annual STD is {stdBNH*100.0:2.2f}")

Average annual return is 0.41%
Average annual STD is 6.40


In [51]:
MOV = movingavg(df, is_plot=False)
avgMOV = np.mean(MOV)
stdMOV = np.std(MOV)

print(f"Average annual return is {avgMOV*100.0:2.2f}%")
print(f"Average annual STD is {stdMOV*100.0:2.2f}")

KeyError: 0

In [42]:
RSI = rsi(df, is_plot=False)
avgRSI = np.mean(RSI)
stdRSI = np.std(RSI)
print(f"Average annual return is {avgRSI*100.0:2.2f}%")
print(f"Average annual STD is {stdRSI*100.0:2.2f}")

KeyError: 0

Question 2: Compute 95% confidence interval of the average annual return for each strategies.

1. Specify the type of statistics used in computing the confidence interval  t-statistics. (z-statistics or t-statistics)

2. 95% Confidence interval for Buy and hold is (-0.00395046907218129, 0.01218092174707841).

3. 95% Confidence interval for moving average crossing is (-0.030241357974889076, 0.030241357974889076).

4. 95% Confidence interval for Bollinger Band is (-0.037909548465146915, 0.037909548465146915).

5. 95% Confidence interval for RSI is (-0.022685857750202515, 0.03091631042509964).


In [25]:
import scipy.stats as st

adj_close = df["Adj Close"]
df_ret = adj_close.pct_change(1).dropna()
num_df = df_ret.count()
df_mean = df_ret.mean()
df_std = df_ret.std()

In [26]:
numBNH = indicator_bnh.count()
intervalBNH = st.t.interval(0.95,df = numBNH - 1, loc = avgBNH, scale = stdBNH/np,sqrt(numBNH))
print(f"95% Confidence interval for moving average crossing is {intervalBNH}")

SyntaxError: positional argument follows keyword argument (<ipython-input-26-3a291d63668d>, line 2)

In [13]:
import scipy.stats as st
import numpy as np
from scipy.stats import norm
from pandas_datareader import data

for stock in set50_list:
    df = data.DataReader(f"{stock}.BK","yahoo","2020-01-01","2020-12-31")
    ['Adj Close']
    indicator = movingavg(df, is_plot=False)

    ret = indicator.pct_change(1).dropna() 
    num = indicator.count()
    mean = indicator.mean()
    std = indicator.std()

    Interval = st.norm.interval(0.95, loc=mean, scale=std/np.sqrt(num))
    print(f"95% Confidence interval for moving average crossing is {Interval}")
    break

95% Confidence interval for moving average crossing is (-0.030241357974889076, 0.030241357974889076)


In [14]:
import scipy.stats as st
import numpy as np
from scipy.stats import norm
from pandas_datareader import data

for stock in set50_list:
    df = data.DataReader(f"{stock}.BK","yahoo","2020-01-01","2020-12-31")
    ['Adj Close']
    indicator = bollinger(df, is_plot=False)

    ret = indicator.pct_change(1).dropna() 
    num = indicator.count()
    mean = indicator.mean()
    std = indicator.std()

    Interval = st.norm.interval(0.95, loc=mean, scale=std/np.sqrt(num))
    print(f"95% Confidence interval for bollinger band is {Interval}")
    break

95% Confidence interval for bollinger band is (-0.037909548465146915, 0.037909548465146915)


In [15]:
import scipy.stats as st
import numpy as np
from scipy.stats import norm
from pandas_datareader import data

for stock in set50_list:
    df = data.DataReader(f"{stock}.BK","yahoo","2020-01-01","2020-12-31")
    ['Adj Close']
    indicator = rsi(df, is_plot=False)

    ret = indicator.pct_change(1).dropna() 
    num = indicator.count()
    mean = indicator.mean()
    std = indicator.std()

    Interval = st.norm.interval(0.95, loc=mean, scale=std/np.sqrt(num))
    print(f"95% Confidence interval for rsi is {Interval}")
    break

95% Confidence interval for rsi is (-0.022685857750202515, 0.03091631042509964)


Use the t-test to make the conclusion whether any of these strategies yield positive return.

1. Buy and Hold has t-score: _________________ and p-value ____________. Can it make profit? ___________ (yes or no). 

2. Moving average crossing has t-score: _________________ and p-value ____________. Can it make profit? ___________ (yes or no). 

3. Bollinger Band has t-score: _________________ and p-value ____________. Can it make profit? ___________ (yes or no). 

4. RSI has t-score: _________________ and p-value ____________. Can it make profit? ___________ (yes or no). 

In [16]:
from scipy.stats import ttest_1samp
# set50_list = ["ADVANC","AOT","AWC","BBL","BDMS","BEM","BGRIM","BH","BJC","BPP","BTS","CBG","CPALL","CPF","CPN","CRC","DTAC","EA","EGCO","GLOBAL","GPSC","GULF","HMPRO","INTUCH","IRPC","IVL","KBANK","KTB","KTC","LH","MINT","MTC","OSP","PTT","PTTEP","PTTGC","RATCH","SAWAD","SCB","SCC","SCGP","TISCO","TMB","TOA","TOP","TRUE","TTW","TU","VGI","WHA"]
set5_list = ["CBG","BGRIM","GULF","OSP","SAWAD"]
stock_info = dict()

for stock in set5_list:
        df = data.DataReader(stock+".BK","yahoo","2020-01-01","2020-12-31")
        ['Adj Close']
        indicator = buyandhold(df, is_plot=False)
        close_price = indicator.values
        ret = close_price[1:]/close_price[:-1] - 1 
        t_score, p_value = ttest_1samp(ret, popmean=0)
        print(f"Buy and Hold of {stock} has t-score: {t_score:2.3f}, and p-value  {p_value:0.4f}.")

Buy and Hold of CBG has t-score: nan, and p-value  nan.
Buy and Hold of BGRIM has t-score: nan, and p-value  nan.
Buy and Hold of GULF has t-score: nan, and p-value  nan.
Buy and Hold of OSP has t-score: nan, and p-value  nan.
Buy and Hold of SAWAD has t-score: nan, and p-value  nan.


Next, we compare againts Buy-and-Hold startegy using t-score. Here,

H0 : A strategy is the same or worst than buy-and-hold

H1: A strategy is better than buy and hold:

1. Moving Average vs Buy and Hold: t-score_______ p-value ______. Is moving average better than buy and hold? _____ (yes or no)

2. Bolling Band vs Buy and Hold: t-score_______ p-value ______. Is moving average better than buy and hold? _____ (yes or no)

3. RSI vs Buy and Hold: t-score_______ p-value ______. Is moving average better than buy and hold? _____ (yes or no)
